In [ ]:
import torchxrayvision as xrv

In [ ]:
import pydicom as dicom
import matplotlib.pylab as plt

In [ ]:
image_path1 = '/home/andrei/Downloads/02aa804e-bde0afdd-112c0b34-7bc16630-4e384014.dcm' 
image_path2 = '/home/andrei/Downloads/174413ec-4ec4c1f7-34ea26b7-c5f994f8-79ef1962.dcm' 

In [ ]:
ds1 = dicom.dcmread(image_path1)
ds2 = dicom.dcmread(image_path2)


In [ ]:
plt.subplot(1,2,1)
plt.imshow(ds1.pixel_array, cmap='gray')
plt.title('02aa804e')

plt.subplot(1,2,2)
plt.imshow(ds2.pixel_array, cmap='gray')
plt.title('174413ec')

FINAL REPORT
 EXAMINATION:  CHEST (PA AND LAT)
 
 INDICATION:  ___F with new onset ascites  // eval for infection
 
 TECHNIQUE:  Chest PA and lateral
 
 COMPARISON:  None.
 
 FINDINGS: 
 
 There is no focal consolidation, pleural effusion or pneumothorax.  Bilateral
 nodular opacities that most likely represent nipple shadows. The
 cardiomediastinal silhouette is normal.  Clips project over the left lung,
 potentially within the breast. The imaged upper abdomen is unremarkable.
 Chronic deformity of the posterior left sixth and seventh ribs are noted.
 
 IMPRESSION: 
 
 No acute cardiopulmonary process.


In [ ]:
folder = '/home/andrei/TUM/SoSe2022/MLMI/mimic-cxr/physionet.org/files/mimic-cxr/2.0.0/files/p10/p10000032/s50414267.txt'
os.path.exists(folder)

In [ ]:
import os
from tqdm import tqdm as tqdm

In [ ]:
MIMIC_CXR_DCM_PATH = '/Users/caghankoksal/Desktop/development/Flamingo-playground/physionet.org/files/mimic-cxr/2.0.0/files/'
MIMIC_CXR_JPG_PATH = '/Users/caghankoksal/Desktop/development/physionet.org/files/mimic-cxr-jpg/2.0.0/files/'
import glob

In [ ]:
# Jpeg files does not include the patient information.
def return_valid_samples(cxr_dcm_path, cxr_jpg_path):

    main_path = cxr_jpg_path

    valid_samples = []
    for folder in tqdm([dir for dir in os.listdir(main_path) if not dir.startswith(".") and not dir.endswith('.html')]):
        #print(f"Cur folder : ",folder)
        for patient in [pat_fold for pat_fold in os.listdir(os.path.join(main_path, folder)) if not pat_fold.endswith('.html') and not pat_fold.startswith(".")]:
            #print("Current patient", patient)
            for record in [cur_file for cur_file in os.listdir(os.path.join(main_path, folder, patient)) if not cur_file.endswith('html') and not cur_file.endswith('.txt') and not cur_file.startswith('.') and cur_file != '/']:
                # Text is only in the record folder
                path_of_record_folder_text = os.path.join(cxr_dcm_path, folder, patient, record)
                path_of_record_folder = os.path.join(main_path, folder, patient, record)
                # Each folder has its corresponding txt file
                txt_path = path_of_record_folder_text + '.txt'
                # Some folders are empty and they are useless
                if len(glob.glob1(path_of_record_folder,"*.jpg")) == 0:
                    #print("No images in this folder",path_of_record_folder)
                    continue
                else:
                    valid_samples.append(
                        {'folder_path': path_of_record_folder,
                        'txt_path': txt_path}
                    )
    return valid_samples

In [ ]:
valid_list = return_valid_samples(MIMIC_CXR_DCM_PATH, MIMIC_CXR_JPG_PATH )

In [ ]:
import json

In [ ]:
with open("../data/external/mimic_cxr_all.json", "w") as final:
    json.dump(valid_list, final, indent=2)
with open("../data/external/valid_data.json", "r") as f:
    valid_data_points = json.load(f)


In [ ]:
valid_data_points

In [ ]:
import torch 
from sklearn import model_selection
train_split, val = model_selection.train_test_split(valid_data_points, test_size=0.2)
val_split, test_split = model_selection.train_test_split(val, test_size=0.5)

In [ ]:
with open("../data/external/mimic_cxr_train.json", "w") as final:
   json.dump(train_split, final, indent=2)

In [ ]:
with open("../data/external/mimic_cxr_val.json", "w") as final:
   json.dump(val_split, final, indent=2)


with open("../data/external/mimic_cxr_test.json", "w") as final:
   json.dump(test_split, final, indent=2)


In [ ]:
len(train_split)

In [ ]:
len(val_split)

In [ ]:
with open("../data/external/valid_data.json", "r") as f:
    valid_data_points = json.load(f)

In [ ]:
from os import makedirs, path, getcwd
from PIL import Image
from tqdm import tqdm as tqdm

In [ ]:
for file in tqdm(valid_data_points):
    folder_path = file['folder_path']
    new_folder_path = folder_path.replace("mimic-cxr-jpg","mimic-cxr-jpg-resized")
    if not path.isdir(new_folder_path):
        makedirs(new_folder_path)

    for img_path in glob.glob1(folder_path,"*.jpg"):
        cur_img_full_path = os.path.join(folder_path, img_path)
        try:
            cur_img = Image.open(cur_img_full_path)
            cur_img_resized = cur_img.resize((224,224))
            cur_img_resized.save(fp=os.path.join(new_folder_path,img_path),format="JPEG",quality=100)
        except:
            print("Error in resizing",cur_img_full_path)
            continue


    

In [ ]:
valid_data_points[0:4]

In [ ]:
import shutil

In [ ]:
def move_all_txt(valid_data_points):
    for file in tqdm(valid_data_points):
        txt_file_path = file['txt_path']
        new_folder_path = '/' + os.path.join(*txt_file_path.split("/")[:-1]).replace("mimic-cxr","mimic-cxr-txt")
        print(new_folder_path)
        if not path.isdir(new_folder_path):
            makedirs(new_folder_path)

        shutil.copyfile(txt_file_path, os.path.join(new_folder_path,txt_file_path.split('/')[-1]))
        
            

In [ ]:
move_all_txt(valid_data_points)

In [ ]:
!pwd

In [1]:
import pandas as pd
import os 

In [2]:
pd.read_csv("/home/mlmi-matthias/physionet.org/files/mimic-cxr-jpg/2.0.0/mimic-cxr-2.0.0-split.csv")

,dicom_id,study_id,subject_id,split
0,02aa804e-bde0afdd-112c0b34-7bc16630-4e384014,50414267,10000032,train
1,174413ec-4ec4c1f7-34ea26b7-c5f994f8-79ef1962,50414267,10000032,train
2,2a2277a9-b0ded155-c0de8eb9-c124d10e-82c5caab,53189527,10000032,train
3,e084de3b-be89b11e-20fe3f9f-9c8d8dfe-4cfd202c,53189527,10000032,train
4,68b5c4b1-227d0485-9cc38c3f-7b84ab51-4b472714,53911762,10000032,train
...,...,...,...,...
377105,428e2c18-5721d8f3-35a05001-36f3d080-9053b83c,57132437,19999733,train
377106,58c403aa-35ff8bd9-73e39f54-8dc9cc5d-e0ec3fa9,57132437,19999733,train
377107,58766883-376a15ce-3b323a28-6af950a0-16b793bd,55368167,19999987,train
377108,7ba273af-3d290f8d-e28d0ab4-484b7a86-7fc12b08,58621812,19999987,train


In [16]:
from tqdm import tqdm as tqdm
import pandas as pd
import os

In [17]:
reports_df = pd.read_csv('/home/mlmi-matthias/Caghan/chexpert/mimic-cxr-2.0.0-chexpert.csv')
reports_df.head()

,subject_id,study_id,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices
0,10000032,50414267,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
1,10000032,53189527,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
2,10000032,53911762,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
3,10000032,56699142,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
4,10000764,57375967,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN


In [18]:
MIMIC_CXR_SPLIT_PATH = '/home/mlmi-matthias/physionet.org/files/mimic-cxr-jpg/2.0.0/mimic-cxr-2.0.0-split.csv'
MIMIC_CXR_METADATA_PATH = '/home/mlmi-matthias/physionet.org/files/mimic-cxr-jpg/2.0.0/mimic-cxr-2.0.0-metadata.csv.gz'

In [19]:
#Read CSVs into pandas dataframes
split_df        = pd.read_csv(MIMIC_CXR_SPLIT_PATH)
metadata_df     = pd.read_csv(MIMIC_CXR_METADATA_PATH)
#reports_df      = pd.read_csv(MIMIC_REPORTS)



In [20]:
metadata_df.head()

,dicom_id,subject_id,study_id,PerformedProcedureStepDescription,ViewPosition,Rows,Columns,StudyDate,StudyTime,ProcedureCodeSequence_CodeMeaning,ViewCodeSequence_CodeMeaning,PatientOrientationCodeSequence_CodeMeaning
0,02aa804e-bde0afdd-112c0b34-7bc16630-4e384014,10000032,50414267,CHEST (PA AND LAT),PA,3056,2544,21800506,213014.531,CHEST (PA AND LAT),postero-anterior,Erect
1,174413ec-4ec4c1f7-34ea26b7-c5f994f8-79ef1962,10000032,50414267,CHEST (PA AND LAT),LATERAL,3056,2544,21800506,213014.531,CHEST (PA AND LAT),lateral,Erect
2,2a2277a9-b0ded155-c0de8eb9-c124d10e-82c5caab,10000032,53189527,CHEST (PA AND LAT),PA,3056,2544,21800626,165500.312,CHEST (PA AND LAT),postero-anterior,Erect
3,e084de3b-be89b11e-20fe3f9f-9c8d8dfe-4cfd202c,10000032,53189527,CHEST (PA AND LAT),LATERAL,3056,2544,21800626,165500.312,CHEST (PA AND LAT),lateral,Erect
4,68b5c4b1-227d0485-9cc38c3f-7b84ab51-4b472714,10000032,53911762,CHEST (PORTABLE AP),AP,2705,2539,21800723,80556.875,CHEST (PORTABLE AP),antero-posterior,NaN


In [21]:
# Combine dataset splitting with metadata
combined = pd.concat((split_df, metadata_df.iloc[:, 3:]), axis=1)

# Use PA or AP view position only
combined = combined.loc[combined['ViewPosition'].isin(['PA', 'AP'])]


In [22]:
combined["split"].unique()

array(['train', 'validate', 'test'], dtype=object)

In [23]:
mimic_root = '/home/mlmi-matthias/physionet.org/'

In [24]:
#report_sectioned = pd.read_csv('/home/mlmi-matthias/Caghan/mimic-cxr-sections/mimic_cxr_sectioned.csv')
#report_sectioned

In [25]:
#report_sectioned.loc[report_sectioned['study'] == "s50414267"]["findings"].values[0]

In [26]:
#combined[""]

In [27]:
#report_sectioned[report_sectioned["study"]=='s'+"51966317"]

In [28]:
reports_sections = pd.read_csv("/home/mlmi-matthias/Caghan/mimic-cxr/txt/sectioned_outputs/mimic_cxr_sections.csv",
            names=["study_id","report"])

In [29]:
reports_sections

,study_id,report
0,s56699142,No acute cardiopulmonary process.
1,s50414267,No acute cardiopulmonary process.
2,s53189527,No acute cardiopulmonary abnormality.
3,s53911762,No acute intrathoracic process.
4,s57375967,"Focal consolidation at the left lung base, pos..."
...,...,...
227830,s58497551,Portable AP radiograph of the chest is demonst...
227831,s57132437,No acute cardiothoracic process.
227832,s58971208,Slight interval worsening of right lower lung ...
227833,s55368167,There has been interval extubation and improve...


In [30]:
# No reports
missing_studies = [
    58235663,
    50798377,
    54168089,
    53071062,
    56724958,
    54231141,
    53607029,
    52035334,
]


for mode in ['validate','test','train']:

    print(f'Processing Dataset: \'{mode}\'')

    df = combined.loc[combined['split'] == mode]
    records = []

    for idx in tqdm(range(len(df))):
        dicom_id, study_id, subject_id, split = df.iloc[idx, :4]
        img_path = f'files/mimic-cxr-jpg/2.0.0/files/p{str(subject_id)[:2]}/p{str(subject_id)}/s{str(study_id)}/{dicom_id}.jpg'

        # Sanity check image exists
        if not os.path.exists(os.path.join(mimic_root, img_path)):
            print(f'IMAGE MISSING: {img_path}')
            continue

        if study_id in missing_studies:
            print(f'{idx}: study [{study_id}] is missing...')
            continue
            
        label = reports_df.loc[reports_df['study_id'] == study_id]
        label.insert(0, 'img_path', img_path)
        
        #print(study_id)
        try:
            matched_report = reports_sections.loc[reports_sections['study_id'] == "s"+str(study_id)]#["findings"]#.values[0]
            
            report = matched_report["report"].values[0]
            #print("matched_report")

            #print

            label.insert(1, 'report', report)
    
        except:
            print("Problem with idx", idx, "study_id : ",study_id)
            print("Problem path : ",img_path)

        records.append(label)
        
    records_df = pd.concat(records)
    #records_df = records_df.drop(['Study'], axis=1)
    records_df.to_csv(f'MIMIC_AP_PA_{mode}.csv', index=False)

        

Processing Dataset: 'validate'


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1959/1959 [00:22<00:00, 88.36it/s]


Processing Dataset: 'test'


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3403/3403 [00:38<00:00, 89.50it/s]


Processing Dataset: 'train'


...

 76%|██████████████████████████████████████████████████████████████████████████████████████████████▎                             | 181103/237972 [33:52<10:26, 90.82it/s]

181091: study [53607029] is missing...


 93%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉        | 222398/237972 [41:42<02:46, 93.79it/s]

222387: study [52035334] is missing...
222388: study [52035334] is missing...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 237972/237972 [44:36<00:00, 88.92it/s]


In [ ]:
pd.read_csv('MIMIC_AP_PA_test.csv')

In [ ]:
from PIL import Image

In [ ]:
Image.open(os.path.join(mimic_root,))